<a href="https://colab.research.google.com/github/Manas-Todi/STYLE-SWAP/blob/main/Task%202/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchsummary import summary
import torch.optim as optim

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
sharp_img = torch.empty(350,3,256,256)
for i,pic in enumerate(sorted(os.listdir("/content/drive/MyDrive/blur_dataset_scaled/sharp"))):
    img=torchvision.io.read_image(path=os.path.join("/content/drive/MyDrive/blur_dataset_scaled/sharp", pic))
    img=torchvision.transforms.Resize((256,256))(img)
    sharp_img[i,:,:,:]=img

In [4]:
motion_blur_img = torch.empty(350,3,256,256)
for i,pic in enumerate(sorted(os.listdir("/content/drive/MyDrive/blur_dataset_scaled/motion_blurred"))):
    img=torchvision.io.read_image(path=os.path.join("/content/drive/MyDrive/blur_dataset_scaled/motion_blurred", pic))
    img=torchvision.transforms.Resize((256,256))(img)
    motion_blur_img[i,:,:,:]=img

In [5]:
defocus_blur_img = torch.empty(350,3,256,256)
for i,pic in enumerate(sorted(os.listdir("/content/drive/MyDrive/blur_dataset_scaled/defocused_blurred"))):
    img=torchvision.io.read_image(path=os.path.join("/content/drive/MyDrive/blur_dataset_scaled/defocused_blurred", pic))
    img=torchvision.transforms.Resize((256,256))(img)
    defocus_blur_img[i,:,:,:]=img

In [6]:
class unet(nn.Module):
	#  Determine what layers and their order in CNN object
    def __init__(self):
        super(unet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3,padding =1)
        self.conv_layer2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3,padding =1)

        self.avg_pool1 = nn.AvgPool2d(kernel_size = 2, stride = 2)

        self.conv_layer3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3,padding =1)
        self.conv_layer4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3,padding =1)

        self.avg_pool2 = nn.AvgPool2d(kernel_size = 2, stride = 2)

        self.conv_layer5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3,padding =1)
        self.conv_layer6 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3,padding =1)

        self.avg_pool3 = nn.AvgPool2d(kernel_size = 2, stride = 2)

        self.conv_layer7= nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3,padding =1)
        self.conv_layer8 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3,padding =1)

        self.avg_pool4 = nn.AvgPool2d(kernel_size = 2, stride = 2)

        self.conv_layer9= nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3,padding =1)
        self.conv_layer10 = nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3,padding =1)

        self.up_conv_1= nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=2,stride=2,padding =0)
        # contatenate here
        self.conv_layer11= nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=3,padding =1)
        self.conv_layer12 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3,padding =1)

        self.up_conv_2= nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=2,stride=2,padding =0)
        # contatenate here
        self.conv_layer13= nn.Conv2d(in_channels=512, out_channels=256, kernel_size=3,padding =1)
        self.conv_layer14= nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3,padding =1)

        self.up_conv_3= nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=2,stride=2,padding =0)
        # contatenate here
        self.conv_layer15= nn.Conv2d(in_channels=256, out_channels=128, kernel_size=3,padding =1)
        self.conv_layer16= nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3,padding =1)

        self.up_conv_4= nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=2,stride=2,padding =0)
        # contatenate here
        self.conv_layer17= nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3,padding =1)
        self.conv_layer18 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3,padding =1)

        self.output = nn.Conv2d(in_channels=64, out_channels=3, kernel_size=1)
    # Progresses data across layers
    def forward(self, x):
        out = self.conv_layer1(x)
        out = F.relu(out)
        gap1 = self.conv_layer2(out)
        out = F.relu(gap1)
        out = self.avg_pool1(out)
        out = self.conv_layer3(out)
        out = F.relu(out)
        gap2 = self.conv_layer4(out)
        out = F.relu(gap2)
        out = self.avg_pool2(out)
        out = self.conv_layer5(out)
        out = F.relu(out)
        gap3 = self.conv_layer6(out)
        out = F.relu(gap3)
        out = self.avg_pool3(out)
        out = self.conv_layer7(out)
        out = F.relu(out)
        gap4 = self.conv_layer8(out)
        out = F.relu(gap4)

        out = self.avg_pool4(out)
        out = self.conv_layer9(out)
        out = F.relu(out)
        out = self.conv_layer10(out)
        out = F.relu(out)

        out=self.up_conv_1(out)
        out=torch.cat((gap4,out),1)
        out = self.conv_layer11(out)
        out = F.relu(out)
        out = self.conv_layer12(out)
        out = F.relu(out)

        out=self.up_conv_2(out)
        out=torch.cat((gap3,out),1)
        out = self.conv_layer13(out)
        out = F.relu(out)
        out = self.conv_layer14(out)
        out = F.relu(out)

        out=self.up_conv_3(out)
        out=torch.cat((gap2,out),1)
        out = self.conv_layer15(out)
        out = F.relu(out)
        out = self.conv_layer16(out)
        out = F.relu(out)

        out=self.up_conv_4(out)
        out=torch.cat((gap1,out),1)
        out = self.conv_layer17(out)
        out = F.relu(out)
        out = self.conv_layer18(out)
        out = F.relu(out)

        out = self.output(out)

        return out

In [7]:
unet= unet()

In [8]:
if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device('cpu')
print("Using {}.".format(device_name))
unet.to(device_name)



Using cuda.


unet(
  (conv_layer1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_layer2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (avg_pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv_layer3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_layer4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (avg_pool2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv_layer5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_layer6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (avg_pool3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv_layer7): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_layer8): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (avg_pool4): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv_layer9): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), paddin

In [9]:
summary(unet, input_size = (3, 256, 256), batch_size = -1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           1,792
            Conv2d-2         [-1, 64, 256, 256]          36,928
         AvgPool2d-3         [-1, 64, 128, 128]               0
            Conv2d-4        [-1, 128, 128, 128]          73,856
            Conv2d-5        [-1, 128, 128, 128]         147,584
         AvgPool2d-6          [-1, 128, 64, 64]               0
            Conv2d-7          [-1, 256, 64, 64]         295,168
            Conv2d-8          [-1, 256, 64, 64]         590,080
         AvgPool2d-9          [-1, 256, 32, 32]               0
           Conv2d-10          [-1, 512, 32, 32]       1,180,160
           Conv2d-11          [-1, 512, 32, 32]       2,359,808
        AvgPool2d-12          [-1, 512, 16, 16]               0
           Conv2d-13         [-1, 1024, 16, 16]       4,719,616
           Conv2d-14         [-1, 1024,

In [10]:
X=torch.cat((motion_blur_img,defocus_blur_img),0)
y=torch.cat((sharp_img,sharp_img),0)
print(X.shape,y.shape)


torch.Size([700, 3, 256, 256]) torch.Size([700, 3, 256, 256])


In [11]:
x_test= X[:600,:,:,:]
x_train=X[600:,:,:,:]
y_test= y[:600,:,:,:]
y_train=y[600:,:,:,:]
print(x_test.shape,x_train.shape)

torch.Size([600, 3, 256, 256]) torch.Size([100, 3, 256, 256])


In [17]:
criterion = nn.MSELoss()
optimizer = optim.Adam(unet.parameters(), lr=0.001)
num_epochs=10
best_loss=10000
for epoch in range(num_epochs):
    i=0
    for data in x_train:
        img=torch.reshape(data, (1,3,256,256)).requires_grad_(True)
        img = img.to(device_name)
        true=y_train[i]
        true=torch.reshape(true,(1,3,256,256)).requires_grad_(True)
        true=true.to(device_name)

        output = unet(img)
        loss = criterion(true, img)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        torch.cuda.empty_cache()
        i+=1

    unet.eval()
    val_loss = 0.0
    with torch.no_grad():
        j=0
        for data in x_test:
            img=torch.reshape(data, (1,3,256,256)).requires_grad_(True)
            img = img.to(device_name)
            true=y_test[j]
            true=torch.reshape(true,(1,3,256,256)).requires_grad_(True)
            true=true.to(device_name)

            output = unet(img)
            loss = criterion(true, img)
            val_loss += loss.item()

            torch.cuda.empty_cache()

    val_loss /= len(x_test)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}')

    if val_loss < best_loss:
        best_loss = val_loss
        best_model_wts = unet.state_dict()

torch.save(best_model_wts, 'best_model.pth')
print('Model with the least loss saved.')

Epoch [1/10], Loss: 6848.7515, Val Loss: 7279.2077
Epoch [2/10], Loss: 6848.7515, Val Loss: 7279.2077
Epoch [3/10], Loss: 6848.7515, Val Loss: 7279.2077
Epoch [4/10], Loss: 6848.7515, Val Loss: 7279.2077
Epoch [5/10], Loss: 6848.7515, Val Loss: 7279.2077
Epoch [6/10], Loss: 6848.7515, Val Loss: 7279.2077
Epoch [7/10], Loss: 6848.7515, Val Loss: 7279.2077
Epoch [8/10], Loss: 6848.7515, Val Loss: 7279.2077
Epoch [9/10], Loss: 6848.7515, Val Loss: 7279.2077
Epoch [10/10], Loss: 6848.7515, Val Loss: 7279.2077
Model with the least loss saved.


In [ ]:
motion_blur_img[0].shape